<a href="https://colab.research.google.com/github/kirbu123/FilmGeneration/blob/develop/FilmGenerationAttemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import usefull libs

In [ ]:
!pip install setuptools-rust

In [ ]:
!pip install transformers==2.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 49.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 16.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This 

In [ ]:
!pip uninstall diffusers

Found existing installation: diffusers 0.11.1
Uninstalling diffusers-0.11.1:
  Would remove:
    /usr/local/bin/diffusers-cli
    /usr/local/lib/python3.10/dist-packages/diffusers-0.11.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/diffusers/*
Proceed (Y/n)? y
  Successfully uninstalled diffusers-0.11.1


In [ ]:
!pip install --upgrade diffusers

In [ ]:
import torch
import time
from datetime import datetime
from tqdm import tqdm
import pandas as pd
from google.colab import drive
import os
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import numpy as np
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, KFold
from sklearn.multiclass import OneVsRestClassifier
from PIL import Image
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd
import csv
from urllib.request import urlopen
import multiprocessing

## Load dataset

Go to root

In [ ]:
!cd /content

Make data directory, go there

In [ ]:
!mkdir laion-high-resolution && cd laion-high-resolution

In [ ]:
# Function for loading 100 rows of dataset from the url
# Param filtering indicate filtering of data by languages (langs) when it is filtering = True
# Param filter_lang sorts data from all langs except filter_lang
# Param file_name sets file_name + .csv
def LoadLaionDataset(filtering = False, filter_lang = '', load_csv = False, file_name = 'dataset', url = 'https://huggingface.co/datasets/laion/laion-high-resolution/viewer/default/train'):
  # Created an URL object in url
  # Create object page
  page = requests.get(url)
  # parser-lxml = Change html to Python friendly format
  # Obtain page's information
  soup = BeautifulSoup(page.text, 'lxml')
  # Obtain information from tag <table>
  table = soup.find('table', attrs={"class": "w-full table-auto rounded-lg font-mono text-xs text-gray-900"})
  # Obtain every title of columns with tag <th>
  headers = []
  for i in table.find_all('th'):
    title = i.text
    headers.append(title[:title.find('\n')])
  # Create a dataframe
  df = pd.DataFrame(columns = headers)
  # Create a for loop to fill mydata
  for j in table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text[:i.text.find('\n')] for i in row_data]
    for i in range(len(row)):
      row[i] = row[i][row[i].find('"') + 1: row[i].rfind('"')]
    length = len(df)
    df.loc[length] = row
  # Filtering by languages
  if filtering:
    df = df[df['LANGUAGE'] == filter_lang]
  # Loading .csv
  if load_csv:
    df.to_csv ('/content/laion-high-resolution/' + str(file_name) + '.csv', index= False )
  return df

In [ ]:
df = LoadLaionDataset(load_csv = True)

In [ ]:
df.head()

,URL,TEXT,WIDTH,HEIGHT,similarity,LANGUAGE,hash,pwatermark,punsafe
0,https://mmedia.ozone.ru/multimedia/1018085734.JPG,"Штора рулонная Эскар, цвет: абрикосовый, ширин...","1,02","1,20",0.27589,ru,"-3,448,109,611,037,364,00",0.05275,0.00129
1,https://images.squarespace-cdn.com/content/v1/...,Elysia Lounge chair from DeLaEspada,"2,50","1,66",0.29183,nolang,"1,517,529,356,012,770,30",0.05878,0.00001
2,https://images.footballfanatics.com/FFImage/th...,Houston Texans Striped Prime Time Backpack,"2,00","2,00",0.39709,en,"5,924,454,315,519,412,00",nul,nul
3,https://images.squarespace-cdn.com/content/v1/...,©Radiohead-Bandcamp,"1,20","1,20",0.26665,nolang,"-3,933,968,474,534,865,40",0.06562,0.00001
4,"https://s3.r29static.com/bin/entry/4ce/0,0,200...",Amazon Registry Best Wedding Gift Items,"2,00","1,05",0.31002,en,"7,893,281,056,273,843,00",0.15549,0.00004


Upgrade of dataset loading function

In [ ]:
# Param frames_count sets count of loading url pages
# Other params are same eith previous function
def LoadData(frames_count = 1, filtering = False, filter_lang = '', load_csv = False, file_name = 'dataset', based_url = 'https://huggingface.co/datasets/laion/laion-high-resolution/viewer/default/train'):
  # DataFrame list
  df_list = list()
  # Updating df_list by loading .csv by url
  for i in tqdm(range(frames_count)):
    df_list.append(LoadLaionDataset(filtering, filter_lang, load_csv = False, url = based_url + '?p=' + str(i)))
  # concating df_list to one big DataFrame
  result_df = pd.concat(df_list)
  X = result_df[['TEXT', 'LANGUAGE']].to_numpy()
  y = result_df['URL'].to_numpy()
  return result_df, list([X, y])

In [ ]:
df, dataset = LoadData(frames_count = 2)

100%|██████████| 2/2 [00:06<00:00,  3.22s/it]


In [ ]:
len(df)

200

In [ ]:
df.head()

,URL,TEXT,WIDTH,HEIGHT,similarity,LANGUAGE,hash,pwatermark,punsafe
0,https://mmedia.ozone.ru/multimedia/1018085734.JPG,"Штора рулонная Эскар, цвет: абрикосовый, ширин...","1,02","1,20",0.27589,ru,"-3,448,109,611,037,364,00",0.05275,0.00129
1,https://images.squarespace-cdn.com/content/v1/...,Elysia Lounge chair from DeLaEspada,"2,50","1,66",0.29183,nolang,"1,517,529,356,012,770,30",0.05878,0.00001
2,https://images.footballfanatics.com/FFImage/th...,Houston Texans Striped Prime Time Backpack,"2,00","2,00",0.39709,en,"5,924,454,315,519,412,00",nul,nul
3,https://images.squarespace-cdn.com/content/v1/...,©Radiohead-Bandcamp,"1,20","1,20",0.26665,nolang,"-3,933,968,474,534,865,40",0.06562,0.00001
4,"https://s3.r29static.com/bin/entry/4ce/0,0,200...",Amazon Registry Best Wedding Gift Items,"2,00","1,05",0.31002,en,"7,893,281,056,273,843,00",0.15549,0.00004


In [ ]:
dataset[0][0]

array(['Штора рулонная Эскар, цвет: абрикосовый, ширина 140 см, высота 170 см81012140170Рулонными шторами Эскар можно оформлять окна как самостоятельно, так и использовать в комбинации с портьерами. Это поможет предотвратить выгорание дорогой ткани на солнце и соединит функционал рулонных с красотой навесных. Преимущества применения рулонных штор для пластиковых окон: - имеют прекрасный внешний вид: многообразие и фактурность материала изделия отлично смотрятся в любом интерьере;- многофункциональны: есть возможность подобрать шторы способные эффективно защитить комнату от солнца, при этом она не будет слишком темной;- есть возможность осуществить быстрый монтаж.ВНИМАНИЕ! Размеры ширины изделия указаны по ширине ткани! Во время эксплуатации не рекомендуется полностью разматывать рулон, чтобы не оторвать ткань от намоточного вала. В случае загрязнения поверхности ткани, чистку шторы проводят одним из способов, в зависимости от типа загрязнения:легкое поверхностное загрязнение можно удал

## KFold sampling on train-test split

In [ ]:
# This func makes KFold from Sklearn more convinient to out task
def KFoldSampling(dataset, nsplits):
  # X, y split from dataset
  X, y = dataset[0], dataset[1]
  kf = KFold(n_splits = nsplits)
  kf.get_n_splits()
  # list of indicies of KFold
  result = list()
  for i, (train_index, test_index) in tqdm(enumerate(kf.split(X))):
    # Appending train-test indicies ro result list
    result.append(np.array([train_index, test_index]))
  return np.array(result)

In [ ]:
KFoldSampling(dataset, nsplits=2)

2it [00:00, 6694.82it/s]


array([[[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
         124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
         136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
         148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
         160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
         172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
         184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
         196, 197, 198, 199],
        [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
          12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,
          24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
          36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,
          48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
          60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
    

## Dataset transform from url to image

In [ ]:
# Function fro loading image
def LoadImageMulti(url, queue):
  response = requests.get(url)
  img = Image.open(BytesIO(response.content))
  queue.put(img)

def LineStop(target, arg, time_limit = 10):
  queue = multiprocessing.Queue()
  p = multiprocessing.Process(target=target, args=(arg, queue))
  p.start()
  p.join(time_limit)
  if p.is_alive():
    print('ABORT')
    # Terminate
    p.terminate()
    return
  return queue.get()

LineStop(LoadImageMulti, 'https://mmedia.ozone.ru/multimedia/1018085734.JPG')

ABORT


In [ ]:
def DatasetTransform(dataset, time_limit = 10, all_time_limit = 1000):
  # Making X, y from dataset
  X = dataset[0]
  y = dataset[1]
  # Making result lists
  result_y = list()
  result_X = list()
  # Start time
  start_time = datetime.now()

  for i, url in enumerate(tqdm(y)):
    # All exceptions are ignored
    try:
      # Load image
      response = requests.get(url)
      img = Image.open(BytesIO(response.content))
    except Exception:
      continue
    # Appending numpy image
    result_y.append(np.array(img))
    result_X.append(X[i])
    # Checking time break
    if (datetime.now() - start_time).seconds > all_time_limit:
      print('Time break')
      break
  return list([np.array(result_X), np.array(result_y)])

In [ ]:
DatasetTransform(dataset, all_time_limit = 20)

  8%|▊         | 15/200 [00:20<04:18,  1.40s/it]

Time break



<ipython-input-43-d17dcedde127>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return list([np.array(result_X), np.array(result_y)])


[array([['Штора рулонная Эскар, цвет: абрикосовый, ширина 140 см, высота 170 см81012140170Рулонными шторами Эскар можно оформлять окна как самостоятельно, так и использовать в комбинации с портьерами. Это поможет предотвратить выгорание дорогой ткани на солнце и соединит функционал рулонных с красотой навесных. Преимущества применения рулонных штор для пластиковых окон: - имеют прекрасный внешний вид: многообразие и фактурность материала изделия отлично смотрятся в любом интерьере;- многофункциональны: есть возможность подобрать шторы способные эффективно защитить комнату от солнца, при этом она не будет слишком темной;- есть возможность осуществить быстрый монтаж.ВНИМАНИЕ! Размеры ширины изделия указаны по ширине ткани! Во время эксплуатации не рекомендуется полностью разматывать рулон, чтобы не оторвать ткань от намоточного вала. В случае загрязнения поверхности ткани, чистку шторы проводят одним из способов, в зависимости от типа загрязнения:легкое поверхностное загрязнение можно уд

## Raw SDXL without finetuning

In [ ]:
from diffusers import DiffusionPipeline, AutoencoderKL

# загрузка SDXL and рафинёра
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix",
                                    torch_dtype=torch.float16)
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)
_ = base.to("cuda")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)
_ = refiner.to("cuda")

# генерация с использованием обеих моделей
high_noise_frac = 0.8
prompt = "an astronaut riding a horse"
negative_prompt = "blurry, bad hands"

image = base(
    prompt=prompt,
    negative_prompt=negative_prompt,
    denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    negative_prompt=negative_prompt,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

ImportError: ignored

## SDXL with Finetuning

In [ ]:
!git clone https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 38605, done.
remote: Counting objects: 100% (9196/9196), done.
remote: Compressing objects: 100% (1570/1570), done.
remote: Total 38605 (delta 8299), reused 8027 (delta 7472), pack-reused 29409
Receiving objects: 100% (38605/38605), 25.55 MiB | 22.95 MiB/s, done.
Resolving deltas: 100% (28557/28557), done.


In [ ]:
!cd diffusers
!pip install -e .

Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!accelerate config

/bin/bash: line 1: accelerate: command not found


In [ ]:
!accelerate config default

/bin/bash: line 1: accelerate: command not found


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.7 MB/s eta 0:00:00


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'/content/dog'

In [ ]:
!export MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export INSTANCE_DIR="dog"
!export OUTPUT_DIR="path-to-save-model"

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --push_to_hub

/usr/bin/python3: can't open file '/content/train_dreambooth.py': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/accelerate_cli.py", line 47, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 986, in launch_command
    simple_launcher(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 628, in simple_launcher
    raise subprocess.CalledProcessError(returncode=process.returncode, cmd=cmd)
subprocess.CalledProcessError: Command '['/usr/bin/python3', 'train_dreambooth.py', '--pretrained_model_name_or_path=', '--instance_data_dir=', '--output_dir=', '--instance_prompt=a photo of sks dog', '--resolution=512', '--train_batch_size=1', '--gradient_accumulation_steps=1', '--learning_rate=5e-6', '--lr_scheduler=constant', '--lr_w

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

ImportError: ignored